In [2]:
from architectures.linear_attributes_autoencoder import Builder as AttrVAEBuilder

In [4]:
attr_vae = AttrVAEBuilder().build(
    128,
    [32, 128, 512]
)

In [5]:
attr_vae

ImageVAE(
  (encoder): LinearEncoder(
    (layers): Sequential(
      (0): Sequential(
        (0): Linear(in_features=32, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (1): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (mu_proj): Linear(in_features=512, out_features=128, bias=True)
    (logvar_proj): Linear(in_features=512, out_features=128, bias=True)
  )
  (decoder): LinearDecoder(
    (latent_proj): Linear(in_features=128, out_features=512, bias=True)
    (decoder_blocks): Sequential(
      (0): Sequential(
        (0): Linear(in_features=512, out_features=128, bias=True)
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplac

# Data preparation

In [ ]:
from 

In [2]:
from utils.audio import (
    load_ogg, 
    get_spectrogram_tensor
)

In [3]:
fourier_params = {
    'fs': 16000,
    'window_size': 2048,
    'window_shift': 1024,
    'type': "hamming"
}

In [10]:
y = load_ogg("data/raw_data/train_audio/21038/iNat65519.ogg", fourier_params['fs'])

In [11]:
X = get_spectrogram_tensor(y, fourier_params)
X.shape

(1025, 5976, 1)